Retrieve Opportunities_examples.csv dataset from DHL.

The columns in this dataset are:
Opp No	Opp Type	Status	Region	Country	BU	Sector	Customer Group	Customer	Product	Created	Cust Dec Date	Go Live Date	Date Exp Gain	Payment Terms	Contract Term	Milestone	 Prob to win % 	BCA_Status	Ann Rev (Ûm)	Ann GP (Ûm)

The model was been generated using Random Forest approach (http://scikit-learn.org/stable/), Pandas (http://pandas.pydata.org/) and Numpy (http://www.numpy.org/).

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (accuracy_score,
                             average_precision_score,
                             auc,
                             #auc_score,
                             classification_report,
                             confusion_matrix,
                             explained_variance_score,
                             f1_score,
                             fbeta_score,
                             hamming_loss,
                             hinge_loss,
                             jaccard_similarity_score,
                             log_loss,
                             matthews_corrcoef,
                             mean_squared_error,
                             mean_absolute_error,
                             precision_recall_curve,
                             precision_recall_fscore_support,
                             precision_score,
                             recall_score,
                             r2_score,
                             roc_auc_score,
                             roc_curve,
                             zero_one_loss)

Sample Opportunities_examples data

In [2]:
train=pd.read_csv('data/Opportunities_examples.csv')
test=pd.read_csv('data/Opportunities_examples.csv')
train['type']='Train' #Create a flag for Train and Test Data set
test['type']='Test'
fullData = pd.concat([train,test],axis=0) #Combined both Train and Test Data set

fullData = fullData.drop(['Opp No','Created', 'Cust Dec Date','Go Live Date','Date Exp Gain','Ann Rev','Ann GP'], axis=1)

fullData.columns # This will show all the column names
fullData.head(10) # Show first 10 records of dataframe

,Opp Type,Status,Region,Country,BU,Sector,Customer Group,Customer,Product,Payment Terms,Contract Term,Milestone,Prob to win %,BCA_Status,type
0,2. Renewal,5. Lost,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,Inbound-to-Manufacturing (I2M),60,< 1 Year,Closed - Lost,0,Approved,Train
1,1. New,5. Lost,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,Aftermarket Services,40,2-3 Years,Closed - Lost,0,Not started,Train
2,1. New,2. Opportunity,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,DHL Lead Logistics Partner,60,2-3 Years,Verbal Customer Commitment Received,90,Awaiting Approval,Train
3,1. New,2. Opportunity,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,DHL Lead Logistics Partner,60,3-5 Years,Qualified and signed off by Sponsor,20,Not started,Train
4,2. Renewal,2. Opportunity,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,Warehousing,60,3-5 Years,Qualified and signed off by Sponsor,85,Awaiting Approval,Train
5,1. New,3. Potential Opps,North America,USA,CLA,Automotive,FORD MOTOR,Ford Motor,NaN,0,1-2 Years,Potential Opportunity,10,NaN,Train
6,2. Renewal,3. Potential Opps,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,DHL Lead Logistics Partner,60,< 1 Year,Potential Opportunity,85,NaN,Train
7,2. Renewal,3. Potential Opps,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Otosan,NaN,0,1-2 Years,Potential Opportunity,85,NaN,Train
8,2. Renewal,1. Gain,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,Warehousing,45,1-2 Years,Contract Signed,100,Approved,Train
9,1. New,7. Cancelled,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,NaN,0,2-3 Years,Closed - Canceled,0,NaN,Train


We want to do is see what data types we have

In [3]:
fullData.dtypes

Opp Type           object
Status             object
Region             object
Country            object
BU                 object
Sector             object
Customer Group     object
Customer           object
Product            object
Payment Terms       int64
Contract Term      object
Milestone          object
 Prob to win %      int64
BCA_Status         object
type               object
dtype: object

Summary of numerical fields

In [4]:
fullData.describe()

,Payment Terms,Prob to win %
count,130.000000,130.000000
mean,51.815385,61.123077
std,27.486370,41.296248
min,0.000000,0.000000
25%,35.000000,10.000000
50%,60.000000,85.000000
75%,60.000000,100.000000
max,120.000000,100.000000


Examples number of each opportunity types

In [5]:
target_col = ["Status"]
cat_cols = ['Opp Type','Region','Country','BU','Sector', 'Customer Group','Customer','Product','Payment Terms','Contract Term','Milestone','BCA_Status']
num_cols= list(set(list(fullData.columns))-set(cat_cols))
other_col=['type'] #Test and Train Data set identifier

fullData.isnull().any()#Will return the feature with True or False,True means have missing value else False

Opp Type           False
Status             False
Region             False
Country            False
BU                 False
Sector             False
Customer Group     False
Customer           False
Product             True
Payment Terms      False
Contract Term      False
Milestone          False
 Prob to win %     False
BCA_Status          True
type               False
dtype: bool

For the sake of simplicity, just fill in the value with the most common value

In [6]:
fullData = fullData.fillna({"Product": "Warehousing"})
fullData = fullData.fillna({"Cust Dec Date": "30/03/2017"})
fullData = fullData.fillna({"Go Live Date": "03/07/2017"})
fullData = fullData.fillna({"BCA_Status": "Approved"})
fullData = fullData.fillna({"Payment Terms": "2-3 Years"})

Combined numerical and Categorical variables.

In [7]:
num_cat_cols = num_cols+cat_cols

Create a new variable for each variable having missing value with VariableName_NA and flag missing value with 1 and other with 0

In [8]:
for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1

Create label encoders for categorical features

In [9]:
for var in cat_cols:
 number = LabelEncoder()
 fullData[var] = number.fit_transform(fullData[var].astype('str'))

In [10]:
fullData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 64
Data columns (total 15 columns):
Opp Type           130 non-null int64
Status             130 non-null object
Region             130 non-null int64
Country            130 non-null int64
BU                 130 non-null int64
Sector             130 non-null int64
Customer Group     130 non-null int64
Customer           130 non-null int64
Product            130 non-null int64
Payment Terms      130 non-null int64
Contract Term      130 non-null int64
Milestone          130 non-null int64
 Prob to win %     130 non-null int64
BCA_Status         130 non-null int64
type               130 non-null object
dtypes: int64(13), object(2)
memory usage: 16.2+ KB


We using Random Forest Classifier to classify opportunity type. 

In [11]:
train=fullData[fullData['type']=='Train']
test=fullData[fullData['type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

features=list(set(list(fullData.columns))-set(target_col)-set(other_col))

x_train = Train[list(features)].values
y_train = Train["Status"].values
x_validate = Validate[list(features)].values
y_validate = Validate["Status"].values
x_test=test[list(features)].values

random.seed(100)
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train, y_train)

status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1], pos_label='T')
roc_auc = auc(fpr, tpr)
print roc_auc

final_status = rf.predict_proba(x_test)
test["Status"]=final_status[:,1]
test.to_csv('model_output.csv',columns=['Status'])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/sklearn/metrics/metrics.py:909: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').